In [2]:
import uproot
import awkward as ak
import mplhep
%matplotlib inline 

import numpy as np
import matplotlib.pyplot as plt

In [3]:
def Effcy_SN(condition,tp):
    indx1 = np.where(condition)[0]
    SN = np.where(mc_type[indx1]==tp)[0].size/len(indx1)
    eff = 0
    aux = {}
    for i in mc_type[indx1]:
        if i==tp: continue
        if i in aux.keys(): aux[i]+=1
        else: aux[i] = 1
    for i in aux.keys(): aux[i]=np.round(100*aux[i]/len(indx1),3)
    if tp=="ELECTRON": eff = 100*np.where(mc_type[indx1]==tp)[0].size/len(mc_ee)
    elif tp=="MUON":   eff = 100*np.where(mc_type[indx1]==tp)[0].size/len(mc_mm)
    elif tp=="TAUS":   eff = 100*np.where(mc_type[indx1]==tp)[0].size/len(mc_tt)
    elif tp=="HADRON": eff = 100*np.where(mc_type[indx1]==tp)[0].size/len(mc_qq)
    return np.array([[eff,SN,aux]])

In [16]:
Effcy_SN(mc_all["E_ecal"]>=75,"ELECTRON")

{'ELECTRON': 90525, 'MUON': 1, 'TAUS': 389, 'HADRON': 3729}


array([[96.50647108,  0.95647902]])

In [4]:
path_mc = './opal_data/mc/'

### Open the file introducing file path
file_ee = uproot.open(path_mc+'ee.root')
file_mm = uproot.open(path_mc+'mm.root')
file_tt = uproot.open(path_mc+'tt.root')
file_qq = uproot.open(path_mc+'qq.root')

ttree_name = 'myTTree'

### Print list of 'branches' of the TTree (i.e. list of variable names)
file_tt[ttree_name].keys()

## Load branches
mc_ee = file_ee[ttree_name].arrays()
mc_mm = file_mm[ttree_name].arrays()
mc_tt = file_tt[ttree_name].arrays()
mc_qq = file_qq[ttree_name].arrays()

mc_ee = ak.to_numpy(mc_ee)
mc_mm = ak.to_numpy(mc_mm)
mc_tt = ak.to_numpy(mc_tt)
mc_qq = ak.to_numpy(mc_qq)

mc_type = np.concatenate((["ELECTRON"]*len(mc_ee),["MUON"]*len(mc_mm),["TAUS"]*len(mc_tt),["HADRON"]*len(mc_qq)))
mc_all = np.concatenate((mc_ee,mc_mm,mc_tt,mc_qq))

In [16]:
#Cuts for ELECTRONS
e_cut = np.empty((0,3))
e_cut = np.append(e_cut,Effcy_SN(mc_all["E_ecal"]>=75,"ELECTRON"),axis=0)
e_cut = np.append(e_cut,Effcy_SN(mc_all["E_hcal"]<=2,"ELECTRON"),axis=0)
e_cut = np.append(e_cut,Effcy_SN((mc_all["E_ecal"]>=65) & (mc_all["Ncharged"]<10),"ELECTRON"),axis=0)
print("Electron: " + str(e_cut))

#Cuts for MUONS
mu_cut = np.empty((0,3))
mu_cut = np.append(mu_cut,Effcy_SN(mc_all["Pcharged"]>=80,"MUON"),axis=0)
mu_cut = np.append(mu_cut,Effcy_SN(mc_all["E_ecal"]<=10,"MUON"),axis=0)
cond = (mc_all["Pcharged"]>=65) & (mc_all["E_ecal"]<75) & (mc_all["Ncharged"]<10)
mu_cut = np.append(mu_cut,Effcy_SN(cond,"MUON"),axis=0)
print("Muon: " + str(mu_cut))

#Cuts for Hadrons
had_cut = np.empty((0,3))
had_cut = np.append(had_cut,Effcy_SN(mc_all["Ncharged"]>=10,"HADRON"),axis=0)
print("Hadron: " + str(had_cut))

#Cuts for TAUS
tau_cut = np.empty((0,3))
cond = (mc_all["Ncharged"]<10) & (mc_all["E_ecal"]>=5) & (mc_all["E_ecal"]<75) & (mc_all["Pcharged"]<80)
tau_cut = np.append(tau_cut,Effcy_SN(cond,"TAUS"),axis=0)
print("Tau: " + str(tau_cut))

Electron: [[96.50647107737575 0.956479016102447
  {'MUON': 0.001, 'TAUS': 0.411, 'HADRON': 3.94}]
 [87.27639069529434 0.6345049409029258
  {'MUON': 13.775, 'TAUS': 20.484, 'HADRON': 2.29}]
 [99.00854992430865 0.9751569751569752
  {'MUON': 0.002, 'TAUS': 1.89, 'HADRON': 0.592}]]
Muon: [[88.61953147349573 0.7271462725494457
  {'ELECTRON': 23.79, 'TAUS': 0.612, 'HADRON': 2.883}]
 [93.2179146226465 0.9127881642562198 {'TAUS': 8.721}]
 [93.04838897659486 0.9479604062995866
  {'ELECTRON': 0.869, 'TAUS': 4.163, 'HADRON': 0.172}]]


Hadron: [[96.69652912350476 0.9984495311927086 {'TAUS': 0.155}]]
Tau: [[93.34461080112101 0.8721838212743872
  {'ELECTRON': 3.213, 'MUON': 5.961, 'HADRON': 3.607}]]
